In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes  # if haven't installed "geopy" yet
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes  # if haven't installed "follium" yet
import folium # plotting library

import datetime
now = datetime.datetime.now()
date = "%4d%02d%02d" % (now.year, now.month, now.day)

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = "RITE5DZBWPGXJKX5S5A2L03WXIAXZOIDDAACBCWIGXO4XAEQ"
CLIENT_SECRET = "HGGFICSXSLIIPMB0QTLVGXVBO3NFMIRKCQFT4PSJDNEJWGOF"
VERSION = date # today's date
limit = 30

In order to convert the *address* to a *(lat, lon)* corrdinate, an agent is needed.
Here we call it "foursquareAPI_agent"

In [3]:
address = "102 North End Ave, New York, NY"
address_nickname = 'Conard Hotel'

geolocator = Nominatim(user_agent = "foursquareAPI_agent") 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

40.7149555 -74.0153365


# 1. search for venue in category
> `https://api.foursquare.com/v2/venues/`**search**`?`
> `client_id=`**CLIENT_ID**`&client_secrete=`**CLIENT_SECRETE**`&v=`**VERSION**
> `&ll=`**LATITUDE`,`LONGITUDE**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [4]:
search_query = "Italian"
radius = 500 # search within 500m radius

### define the query url

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,VERSION, search_query, radius, limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=RITE5DZBWPGXJKX5S5A2L03WXIAXZOIDDAACBCWIGXO4XAEQ&client_secret=HGGFICSXSLIIPMB0QTLVGXVBO3NFMIRKCQFT4PSJDNEJWGOF&ll=40.7149555,-74.0153365&v=20190321&query=Italian&radius=500&limit=30'

### send the GET request and examine the result

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c93f1be4c1f6729379bc203'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294544',
     'url': 'https://www.seamless.com/menu/harrys-italian-pizz

### transform the result into pandas dataframe

In [7]:
# take relevant results
venues = results['response']['venues']

# transform into pandas dataframe
df = json_normalize(venues)
df.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1553199550
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1553199550
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1553199550


In [8]:
# a function we will use for extracting the category name form the lengthy category feedback

def get_category_list(row):
    try: 
        category_list = row['categories']
    except:
        category_list = row['venues.categories']
    
    if len(category_list) == 0:
        return None
    else: 
        return category_list[0]['name']

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_col = ['name','id','categories'] + [col for col in df.columns if col.startswith('location.')]
df = df.loc[:,filtered_col]

# simplify the category col
df['categories'] = df.apply(get_category_list, axis = 1)

# simplify the column index (getting rid of all the "location.")
df.columns = [col.split(".")[-1] for col in df.columns]

df.head()

,name,id,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state
0,Harry's Italian Pizza Bar,4fa862b3e4b0ebff2f749f06,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY
1,Conca Cucina Italian Restaurant,4f3232e219836c91c7bfde94,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY
2,Ecco,3fd66200f964a520f4e41ee3,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY


### generate visualization using folium

In [11]:
venue_map = folium.Map(location = [latitude, longitude], zoom_start = 16) # generate a map centered around address

# add red circle representing the centered address
folium.features.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color = 'red',
    popup = address_nickname,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6    
    ).add_to(venue_map)

# add the searched results as blue circles
for lat, lon, cat, name in zip(df.lat, df.lng, df.categories, df.name):
    label = "{} ({})".format(name, cat)           # popup only takes raw html format
    label = folium.Popup(label, parse_html=True)  # when the string has ' or &, need parse_html=True
    folium.features.CircleMarker(
        [lat, lon],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6
        ).add_to(venue_map)

# display map
venue_map

# 2. Explore a given venue
## 2.1 get more details for this venue
> (premium call)
>
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?`
> `client_id=`**CLIENT_ID**`&client_secrete=`**CLIENT_SECRETE**`&v=`**VERSION**

## 2.2 get tips for this venue
> (regular call)
>
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**/tips`?`
> `client_id=`**CLIENT_ID**`&client_secrete=`**CLIENT_SECRETE**`&v=`**VERSION**`&limit=`**LIMIT**

### make a call for specific venue

In [39]:
# a function that generates url to look into the venue
def venue_detail_url(client_id, client_secret, version,id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id,client_id,client_secret, version)
    return url

def venue_tips_url(client_id, client_secret, version,id,limit):
    url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(id,client_id,client_secret, version, limit)
    return url

# a funtion that extracts the rating after call is made
def rating(results):
    try: 
        return results['response']['venue']['rating']
    except:
        return "This venue does not have rating yet."

In [30]:
venue_id = [id for id in df.id]

In [31]:
url = venue_detail_url(CLIENT_ID, CLIENT_SECRET, VERSION, venue_id[0])
result = requests.get(url).json()

# these are the info we can retrieve for each venue
print(result['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


### check ratings for all venues

In [34]:
for i in range(len(venue_id)):
    url = venue_detail_url(CLIENT_ID, CLIENT_SECRET, VERSION, venue_id[i])
    result = requests.get(url).json()
    print(result['response']['venue']['name'],":", rating(result),"\n")

Harry's Italian Pizza Bar : 7.1 

Conca Cucina Italian Restaurant : This venue does not have rating yet. 

Ecco : 7.8 



### get the number of tips
the third one seems to have the highest rating, let's explore its tips

In [38]:
result['response']['venue']['tips']['count']

16

In [43]:
limit = 10
url = venue_tips_url(CLIENT_ID,CLIENT_SECRET,VERSION,venue_id[2],limit)

In [50]:
tips_result = requests.get(url).json()
tips_result

{'meta': {'code': 200, 'requestId': '5c93fbc31ed2192729f4c74b'},
 'response': {'tips': {'count': 16,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 2,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'El-Tawil',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}},
     'authorInteractionType': 'liked'},
    {'id': '5247cb68498eeac13eb4c3e1',
    

In [53]:
tips = tips_result['response']['tips']['items']

tip =tips[0]
tip.keys()
tip

{'id': '5ab1cb46c9a517174651d3fe',
 'createdAt': 1521601350,
 'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
 'type': 'user',
 'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
 'lang': 'en',
 'likes': {'count': 0, 'groups': []},
 'logView': True,
 'agreeCount': 2,
 'disagreeCount': 0,
 'todo': {'count': 0},
 'user': {'id': '484542633',
  'firstName': 'Nick',
  'lastName': 'El-Tawil',
  'gender': 'male',
  'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
   'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}},
 'authorInteractionType': 'liked'}

In [54]:
pd.set_option('display.max_colwidth', -1) # set the df width to the max of the page

tips_df = json_normalize(tips)

tips_filtered_col = ['text','agreeCount','disagreeCount','id','user.firstName','user.lastName','user.gender','user.id']
tips_df = tips_df.loc[:,tips_filtered_col]
tips_df

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",2,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633
1,"Service was truly excellent- attentive, quick, & friendly. Food was equally amazing- particularly calamari and pollo alla scarpiello. The sauces were incredibly flavorful. Great wine list too! A+++",0,0,5247cb68498eeac13eb4c3e1,Nicole,F,female,25279241


note that we only have 2 instead of 15 tips, becuase we have limited access with this account

# 3. search related to a Foursquare user
### 3.1 get more detail for user
> `https://api.foursquare.com/v2/`**users**`/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### 3.2 get the tips for this user
> `https://api.foursquare.com/v2/`**users**`/`**USER_ID**`/`**tips**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

### 3.3 get a specific tip
> `https://api.foursquare.com/v2/`**tips**`/`**TIPS_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

In [82]:
user_id = 484542633

# user detail
def user_detail_url(client_id, client_secret,version,user_id):
    url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id,client_id,client_secret, version)
    return url

# user tips
def user_tips_url(client_id, client_secret,version,user_id,limit):
    url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id,client_id,client_secret, version,limit)
    return url

# a specific tip
def tip_url(client_id, client_secret,version,tip_id):
    url = 'https://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id,client_id,client_secret, version)
    return url


### get details for a user

In [57]:
url = user_detail_url(CLIENT_ID,CLIENT_SECRET,VERSION,user_id)
results = requests.get(url).json()
user_data = results['response']['user']
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [66]:
print(user_data['firstName'],user_data['lastName'], "(from", user_data['homeCity'],")")
user_data['tips']

Nick El-Tawil (from New York, NY )


{'count': 240}

### get tips from this user

In [81]:
limit = 15
url = user_tips_url(CLIENT_ID,CLIENT_SECRET,VERSION,user_id,limit)
results = requests.get(url).json()

user_tips = results['response']['tips']['items']

pd.set_option('display.max_colwidth', -1)
tips_df = json_normalize(user_tips)

filtered_col = ['text','agreeCount','disagreeCount','id']
tips_df = tips_df.loc[:,filtered_col]
tips_df

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
8,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
9,Coffee game on point,1,0,5acbbb1501235808d5d6525e


### get the tip that's been agreed the most

In [91]:
tip_id = '5ab5575d73fe2516ad8f363b'

url = tip_url(CLIENT_ID,CLIENT_SECRET,VERSION,tip_id)
results = requests.get(url).json()

# the venue corresponding to the most agreed tip
tip = results['response']['tip']['venue']
tip.keys()

dict_keys(['id', 'name', 'location', 'categories', 'delivery', 'photos', 'venuePage'])

In [94]:
print(tip['name'])
print(tip['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


### user's friend

In [97]:
friends = user_data['friends']
friends

{'count': 0,
 'groups': [{'type': 'others',
   'name': 'Other friends',
   'count': 0,
   'items': []}]}

hmmmm he does not have friends on FourSquare

### user's profile picture

In [99]:
user_data['photo']

{'prefix': 'https://fastly.4sqi.net/img/user/',
 'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}

In [103]:
photo_url = "{}{}".format(user_data['photo']['prefix'][:-1:],user_data['photo']['suffix'])
print(photo_url)

https://fastly.4sqi.net/img/user/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg


In [106]:
Image(url = photo_url)

# 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

In [107]:
def venue_explore_url(client_id, client_secret,lat, lon, version, limit):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(client_id,client_secret, version,lat,lon,limit)
    return url

In [111]:
limit = 15
url = venue_explore_url(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION, limit)
results = requests.get(url).json()

In [125]:
# results['response']['groups'][0]['items']  ==> groups(group number of recommandations, only 1 group)
#                                            ==> items(list of recommandations)
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4a6fa0abf964a520a4d61fe3',
  'name': 'Brookfield Place - BFPL',
  'location': {'address': '230 Vesey St',
   'crossStreet': 'at West St',
   'lat': 40.713203629795636,
   'lng': -74.01561859633297,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.713203629795636,
     'lng': -74.01561859633297}],
   'distance': 196,
   'postalCode': '10281',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['230 Vesey St (at West St)',
    'New York, NY 10281',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1fd941735',
    'name': 'Shopping Mall',
    'pluralName': 'Shopping Malls',
    'shortName': 'Mall',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'grou

In [141]:
# clean up the recommendation list
df = json_normalize(items)
filtered_col = ['venue.name', 'venue.categories','venue.id'] + [col for col in df.columns if col.startswith('venue.location.')]
df = df.loc[:, filtered_col]

# remove venue.location
df.columns = [col.split('.')[-1] for col in df.columns]

# extract category
df['categories'] = df.apply(get_category_list, axis = 1)
df.head()


,name,categories,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state
0,Brookfield Place - BFPL,Shopping Mall,4a6fa0abf964a520a4d61fe3,230 Vesey St,US,New York,United States,at West St,196,"[230 Vesey St (at West St), New York, NY 10281, United States]","[{'label': 'display', 'lat': 40.713203629795636, 'lng': -74.01561859633297}]",40.713204,-74.015619,NaN,10281,NY
1,"Waterfront Plaza, Brookfield Place",Plaza,4a0b04dcf964a520ba741fe3,230 Vesey Street,US,New York,United States,NaN,205,"[230 Vesey Street, New York, NY 10281, United States]","[{'label': 'display', 'lat': 40.71324098127375, 'lng': -74.01624118660538}]",40.713241,-74.016241,NaN,10281,NY
2,Asphalt Green Battery Park City,Athletics & Sports,51bcd723498eecb51dd996bc,212 N End Ave,US,New York,United States,Murray Street,121,"[212 N End Ave (Murray Street), New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.715923644223366, 'lng': -74.01467345576101}]",40.715924,-74.014673,NaN,10282,NY
3,Irish Hunger Memorial,Monument / Landmark,4a397f7ef964a520839f1fe3,Vesey St & North End Ave,US,New York,United States,NaN,104,"[Vesey St & North End Ave, New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.71513873128946, 'lng': -74.01655655956712}]",40.715139,-74.016557,Battery Park City,10282,NY
4,Shake Shack,Burger Joint,4dda977ad22d38ef4332c419,215 Murray St,US,New York,United States,btwn West St & North End Ave,64,"[215 Murray St (btwn West St & North End Ave), New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.71532572032048, 'lng': -74.01475032559945}]",40.715326,-74.014750,NaN,10282,NY


In [143]:
# plot it in a folium map

rec_map = folium.Map(location = [latitude, longitude], zoom_start = 16) # generate a map centered around address

# add red circle representing the centered address
folium.features.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color = 'red',
    popup = address_nickname,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6    
    ).add_to(rec_map)

# add the searched results as blue circles
for lat, lon, cat, name in zip(df.lat, df.lng, df.categories, df.name):
    label = "{} ({})".format(name, cat)           # popup only takes raw html format
    label = folium.Popup(label, parse_html=True)  # when the string has ' or &, need parse_html=True
    folium.features.CircleMarker(
        [lat, lon],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6
        ).add_to(rec_map)

# display map
rec_map

# 5. Explore Trending Venue
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

In [149]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results['response']['venues']
        

[{'id': '4eefd4506d86d722df8fd52e',
  'name': 'Houston Hall',
  'location': {'address': '222 W Houston St',
   'crossStreet': 'btwn Varick & Bedford St',
   'lat': 40.72879213466685,
   'lng': -74.00476202123185,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.72879213466685,
     'lng': -74.00476202123185}],
   'distance': 1779,
   'postalCode': '10014',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['222 W Houston St (btwn Varick & Bedford St)',
    'New York, NY 10014',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d117941735',
    'name': 'Beer Garden',
    'pluralName': 'Beer Gardens',
    'shortName': 'Beer Garden',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/beergarden_',
     'suffix': '.png'},
    'primary': True}],
  'venuePage': {'id': '49129782'}}]

In [ ]:
if len(results['response']['venues']==0):
    print("Currently, there is no trending venue.")
else: trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)